# Unsteady Stokes problem

We consider the unsteady Stokes problem defined in the domain $\Omega := {[0,1]}^2$ by the following system of PDEs, boundary conditions, and initial conditions: 
$$
\begin{cases}
  \frac{\partial \boldsymbol{u}}{\partial t} 
  -\mu\boldsymbol{\Delta} \boldsymbol{u} + \boldsymbol{\nabla} p = \boldsymbol{f}, 
  \qquad&\text{in }\Omega\times(0,1), \\
  \boldsymbol{\nabla} \cdot \boldsymbol{u} = 0, 
  \qquad &\text{in }\Omega\times(0,1), \\
  \boldsymbol{u} = \boldsymbol{u}_D \;\; &{\rm on} \ \Gamma_D\times(0,1), \\
  (\mu\boldsymbol{\nabla} \boldsymbol{u} - p)\cdot\boldsymbol{n} = \boldsymbol{g}_N\;\; 
  &{\rm on}\ \partial\Omega\setminus \Gamma_D\times(0,1), \\
  \boldsymbol{u}(\cdot,t=0) = \boldsymbol{u}_0, &\text{in }\Omega. 
\end{cases}
$$

In the following test case we consider $\Gamma_D = \{\boldsymbol{x}\in\partial\Omega\; |\; x_1 = 0\} \cup \{\boldsymbol{x}\in\partial\Omega\; |\; x_2 = 0\}.$ 


## Saddle-point weak formulation (primal form)

The weak formulation is derived by testing the first equation with a vector-valued function $\boldsymbol{v}$ in $H^1_{0,\Gamma_D}(\Omega)^d$ and the second equation with $q\in L^2(\Omega)$. Assuming, for the sake of simplicity, that $\boldsymbol{u}_D = \boldsymbol{0}$, and integrating by parts in the first equation, we end up with the saddle point problem
\begin{align*}
\int_{\Omega} \frac{\partial\boldsymbol{u}}{\partial t} \cdot \boldsymbol{v} +
\int_{\Omega} \mu \boldsymbol{\nabla} \boldsymbol{u} \cdot \boldsymbol{\nabla} \boldsymbol{v}
- (\boldsymbol{\nabla} \cdot \boldsymbol{v}) \ p \ {\rm d} x
&= \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \, {\rm d} x + \int_{\Gamma_N} \boldsymbol{g}_N \cdot \boldsymbol{v} \, {\rm d} s. \\
\int_\Omega (\boldsymbol{\nabla} \cdot \boldsymbol{u}) \ q \ {\rm d} x &= 0
\end{align*}

Defining $a: H^1_{0,\Gamma_D}(\Omega)^d\times H^1_{0,\Gamma_D}(\Omega)^d\to\mathbb{R}$ and $b:H^1_{0,\Gamma_D}(\Omega)^d\times L^2(\Omega)\to\mathbb{R}$ such that
\begin{align*}
a(\boldsymbol{v},\boldsymbol{w}) &:=\int_{\Omega} \mu \boldsymbol{\nabla} \boldsymbol{v} \cdot \boldsymbol{\nabla} \boldsymbol{w}\  {\rm d} x, 
\quad\forall\ v,w\in H^1_{0,\Gamma_D}(\Omega)^d \\ 
b(\boldsymbol{v},q) &:= -\int_\Omega (\boldsymbol{\nabla} \cdot \boldsymbol{v}) \ q \ {\rm d} x, \quad\forall\ v\in H^1_{0,\Gamma_D}(\Omega)^d \;\text{and}\; \forall\ q\in L^2(\Omega),
\end{align*}

the weak form of Stokes problem reads: For all $t\in (0,1)$, find $(u,p)(t)$ such that

\begin{align*}
\left(\frac{\partial\boldsymbol{u}}{\partial t}, \boldsymbol{v}\right)_\Omega +
a(\boldsymbol{u},\boldsymbol{v}) + b(\boldsymbol{v},p)
&= \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \, {\rm d} x + \int_{\Gamma_N} \boldsymbol{g}_N \cdot \boldsymbol{v} \, {\rm d} s. \\
b(\boldsymbol{u},q) &= 0.
\end{align*}

## Numerical solution with discontinuous Galerkin method in FEniCS I

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def dg_stokes(n, deg, dt, mu, u_exact, p_exact, f, gNt, gNr):    
    # 1. generate the mesh and mark the boundaries
    # mesh = UnitSquareMesh(n, n, 'crossed')
    square = Rectangle(Point(0.0, 0.0), Point(1.0, 1.0))
    mesh = generate_mesh(square, n)
    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space
    V = VectorElement('DG', mesh.ufl_cell(), deg)
    Q = FiniteElement('DG', mesh.ufl_cell(), deg-1)
    X = FunctionSpace(mesh, V * Q)

    # 3. problem definition
    u, p = TrialFunctions(X)
    v, q = TestFunctions(X)
    x_old = Function(X)
    u_old, p_old = split(x_old)
    
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    h_avg = (h('+') + h('-'))/2
    alpha = 10.1*mu*deg**2
    beta = 2.1/deg

    a = (dot(u, v)/Constant(dt))*dx  \
      + (mu*inner(grad(u), grad(v)) - p*div(v) + div(u)*q)*dx  \
      - inner(jump(u, n), avg(q))*dS + inner(jump(v, n), avg(p))*dS \
      - inner(u, n)*q*ds(3) + inner(v, n)*p*ds(3) \
      - inner(dot(avg(mu*grad(u)), n('+')), jump(v))*dS - inner(jump(u), dot(mu*avg(grad(v)), n('+')))*dS \
      - inner(dot(mu*grad(u), n), v)*ds(3) - inner(u, dot(mu*grad(v), n))*ds(3) \
      + alpha/h_avg*inner(jump(u), jump(v))*dS + alpha/h*inner(u, v)*ds(3) \
      + beta*h_avg*jump(p)*jump(q)*dS 

    L = (dot(u_old, v)/Constant(dt)) * dx + dot(f, v) * dx + dot(gNt, v) * ds(1) + dot(gNr, v) * ds(2)
    
    # 4. solution
    A = assemble(a)
    Ainv = LUSolver(A)

    time = 0.0
    max_eH1 = 0.0
    max_eL2 = 0.0

    while time < 1.0:
        time += dt
        f.t = time
        gNt.t = time
        gNr.t = time
        u_exact.t = time
        p_exact.t = time
        
        b = assemble(L)
        Ainv.solve(x_old.vector(), b)
        u_old, p_old = x_old.split()
  
        eH1 = errornorm(u_exact, u_old, 'H1')
        eL2 = errornorm(p_exact, p_old, 'L2')
        #print('time={:.2} eH1={:.3e} eL2={:.3e}'.format(time, eH1, eL2))
  
        max_eH1 = max(max_eH1, eH1)
        max_eL2 = max(max_eL2, eL2)
    
    return max_eH1, max_eL2

## Exact Solution for convergence test I

Compute the functions $\boldsymbol{f}$, $\boldsymbol{g}_N$, $\boldsymbol{u}_D$ and $\boldsymbol{u}_0$ corresponding to the analytical solution
$$
            \boldsymbol{u}(x,y) = \sin(2t) \begin{bmatrix}(1-\cos(\pi x))\sin(\pi y) \\ \sin(\pi x)(\cos(\pi y)-1)\end{bmatrix},
            \qquad
            p(x,y) = -\frac{\sin(2t)}{4}(\cos{(2\pi x)}+\cos{(2\pi y)}).
$$

Assess the convergence, measuring the errors as 
$$
{\rm err}_u = \max_{t\in(0,1]}||\boldsymbol{u} - \boldsymbol{u}_h||_{H^1(\Omega)}, 
\quad\text{ and}\quad {\rm err}_p = \max_{t\in(0,1]}||p - p_h||_ {L^2(\Omega)}.
$$

In [ ]:
u_exact = Expression((
    '(1.0-cos(pi*x[0])) * sin(pi*x[1]) * sin(2 * t)',
    'sin(pi*x[0]) * (cos(pi*x[1])-1.0) * sin(2 * t)'
), degree=4, t=0.0)
p_exact = Expression(
    '-0.25 * (cos(2*pi*x[0]) + cos(2*pi*x[1])) * sin(2 * t)',
    degree=4, t=0.0)

mu = Constant(1.0)
f = Expression((
        'mu*pi*pi*sin(pi*x[1])*(1-2*cos(pi*x[0]))*sin(2*t) + 0.5*pi*sin(2*pi*x[0])*sin(2*t) + 2*(1-cos(pi*x[0]))*sin(pi*x[1])*cos(2*t)',
        'mu*pi*pi*sin(pi*x[0])*(2*cos(pi*x[1])-1)*sin(2*t) + 0.5*pi*sin(2*pi*x[1])*sin(2*t) + 2*sin(pi*x[0])*(cos(pi*x[1])-1)*cos(2*t)'
               ), degree=2, t=0.0, mu=mu)
gNt = Expression(('mu * pi * (cos(pi*x[0])-1.0) * sin(2*t)', '0.25 * (cos(2*pi*x[0]) + cos(2*pi*x[1])) * sin(2*t)'
                 ), degree=2, t=0.0, mu=mu)
gNr = Expression(('0.25 * (cos(2*pi*x[0]) + cos(2*pi*x[1])) * sin(2*t)', 'mu * pi * (1.0-cos(pi*x[1])) * sin(2*t)'
                 ), degree=2, t=0.0, mu=mu)

degree = 1
dt = 0.02
for n in [8, 16, 32, 64]:
    euH1, epL2 = dg_stokes(n, degree, dt, mu, u_exact, p_exact, f, gNt, gNr)
    print('errors: u_H1={:.3e} p_L2={:.3e}'.format(euH1, epL2))

errors: u_H1=4.509e-01 p_L2=1.113e-01
errors: u_H1=2.253e-01 p_L2=4.801e-02
errors: u_H1=1.153e-01 p_L2=1.850e-02
errors: u_H1=5.943e-02 p_L2=1.023e-02


## Pseudostress weak formulation (dual form)

We rewrite the Stokes problem in a different form by introducing the stress $\boldsymbol{\sigma}(\boldsymbol{u},p) = \mu\boldsymbol{\nabla}\boldsymbol{u} - p\mathbb{I}_d$ as additional variable. Then, the Stokes system reads

$$
\begin{cases}
  \frac{\partial \boldsymbol{u}}{\partial t} 
  -\boldsymbol{\nabla}\cdot\boldsymbol{\sigma} = \boldsymbol{f}, 
  \qquad&\text{in }\Omega\times(0,1), \\
  \mu^{-1}{\rm dev}(\boldsymbol{\sigma}) - \boldsymbol{\nabla}\boldsymbol{u} = 0, 
  \qquad &\text{in }\Omega\times(0,1), \\
  \boldsymbol{u} = \boldsymbol{u}_D \;\; &{\rm on} \ \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}\cdot\boldsymbol{n} = \boldsymbol{g}_N\;\; 
  &{\rm on}\ \partial\Omega\setminus \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}(\cdot,t=0) = \boldsymbol{\sigma}_0, &\text{in }\Omega, 
\end{cases}
$$

where the deviatioric operator is defined such that 
${\rm dev}(\boldsymbol{\tau}) = \boldsymbol{\tau} - \frac1d {\rm tr}(\boldsymbol{\tau})\mathbb{I}_d$.
Notice that the incompressibility constraint $\boldsymbol{\nabla}\cdot\boldsymbol{u} = 0$ is enforced through the second equation of the previous system that yields ${\rm tr}(\boldsymbol{\nabla}\boldsymbol{u}) = \boldsymbol{\nabla}\cdot\boldsymbol{u} = 0$. 

We remark that we have also replaced the initial condition on $\boldsymbol{u}(\cdot,t=0)$ with a condition on $\boldsymbol{\sigma}(\cdot,t=0)$. This can be done under the assumption that the velocity and pressure solution are sufficiently regular. Finally, we point out that the pressure can be recoverd from the stress according to $p = -\frac1d {\rm tr}(\boldsymbol{\sigma})$.

Assuming enough regularity of the problem data and solution, we can derive in time the second and third equations and plug the expression of $\frac{\partial \boldsymbol{u}}{\partial t}$ that we get from the first equation into the second one. Thus, we obtain

$$
\begin{cases}
  \mu^{-1}\frac{\partial}{\partial t} {\rm dev}(\boldsymbol{\sigma})
  - \boldsymbol{\nabla}\left(\boldsymbol{\nabla}\cdot\boldsymbol{\sigma}\right)=\boldsymbol{F}, 
  \qquad &\text{in }\Omega\times(0,1), \\
  \boldsymbol{\nabla}\cdot\boldsymbol{\sigma}=\boldsymbol{g}_D \;\; &{\rm on}\ \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}\cdot\boldsymbol{n} = \boldsymbol{g}_N\;\; 
  &{\rm on}\ \partial\Omega\setminus \Gamma_D\times(0,1), \\
  \boldsymbol{\sigma}(\cdot,t=0) = \boldsymbol{\sigma}_0, &\text{in }\Omega, 
\end{cases}
$$

with $\boldsymbol{F}= \boldsymbol{\nabla}\boldsymbol{f}$ and $\boldsymbol{g}_D = \frac{\partial\boldsymbol{u}_D}{\partial t}-\boldsymbol{f}$. In this way, we reformulated the problem only in the pseudostress variable $\boldsymbol{\sigma}$.

For simplicity, we assume that $\boldsymbol{g}_N=\boldsymbol{0}$, so that we can define the functional space for the pseudostress tensor field $\boldsymbol{\sigma}$ as $$\boldsymbol{H}_{0,\Gamma_n}({\rm div}, \Omega)^d = \{\boldsymbol{\eta}\in\boldsymbol{H}({\rm div}, \Omega)^d \;|\; \boldsymbol{\eta}\cdot\boldsymbol{n}=0 \; \text{on}\; \Gamma_N\}.$$ 

Testing the first equation with $\boldsymbol{\tau}\in \boldsymbol{H}_{0,\Gamma_n}({\rm div}, \Omega)^d$ and integrating by parts we obtain the following dual weak formulation of the Stokes problem:

\begin{align*}
\int_{\Omega} \mu^{-1} \frac{\partial{\rm dev}(\boldsymbol{\sigma})}{\partial t} : {\rm dev}(\boldsymbol{\tau}) 
+ \int_{\Omega} (\boldsymbol{\nabla}\cdot\boldsymbol{\sigma}) \cdot (\boldsymbol{\nabla}\cdot\boldsymbol{\tau})
\ {\rm d} x
    &= \int_{\Omega} \boldsymbol{F} : \boldsymbol{\tau} \, {\rm d} x 
    + \int_{\Gamma_D} \boldsymbol{g}_D \cdot \boldsymbol{\tau}\boldsymbol{n} \, {\rm d} s, 
\end{align*}

## Numerical solution with discontinuous Galerkin method in FEniCS II

In [ ]:
def dg_psstokes(n, deg, dt, mu, s_exact, f, gD):    
    # 1. generate the mesh and mark the boundaries
    square = Rectangle(Point(0.0, 0.0), Point(1.0, 1.0))
    mesh = generate_mesh(square, n)    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)
    
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space
    S = TensorFunctionSpace(mesh, 'DG', deg)

    # 3. problem definition
    s = TrialFunction(S)
    r = TestFunction(S)
    s_old = Function(S)
    
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    h_avg = (h('+') + h('-'))/2
    alpha = 10.1*deg**2

    a = inner(dev(s), dev(r))/(mu*Constant(dt)) * dx + dot(div(s), div(r)) * dx  \
      - dot(avg(div(s)), jump(r, n)) * dS - dot(jump(s, n), avg(div(r))) * dS \
      - dot(div(s), dot(r, n)) * ds(3) - dot(dot(s, n), div(r)) * ds(3) \
      + alpha/h_avg * dot(jump(s, n), jump(r, n)) * dS \
      + alpha/h*dot(dot(s, n), dot(r, n)) * ds(3)  

    L = (inner(dev(s_old), dev(r))/(mu*Constant(dt)) + inner(f, r)) * dx + dot(gD, dot(r, n)) * (ds(1)+ds(2))
    
    # 4. solution
    A = assemble(a)
    Ainv = LUSolver(A)

    time = 0.0
    max_eHd = 0.0
    max_eL2 = 0.0

    while time < 1.0:
        time += dt
        f.t = time
        gD.t = time
        s_exact.t = time       
        b = assemble(L)
        Ainv.solve(s_old.vector(), b)
  
        eHd = errornorm(s_exact, s_old, 'Hdiv0')
        eL2 = errornorm(s_exact, s_old, 'L2')
        max_eHd = max(max_eHd, eHd)
        max_eL2 = max(max_eL2, eL2)
    
    return max_eHd, max_eL2

## Exact Solution for convergence test II

Compute the functions $\boldsymbol{F}$, $\boldsymbol{g}_D$, $\boldsymbol{g}_N$ and $\boldsymbol{\sigma}_0$ corresponding to the analytical solution
$$
            \boldsymbol{\sigma}(x,y) = \sin(2t) \begin{bmatrix}\sin(\pi x)\sin(\pi y) & 0 \\ 0 & -\sin(\pi x)\sin(\pi y)\end{bmatrix}.           
$$

Assess the convergence, measuring the errors as 
$$
{\rm err}_{\rm div} = \max_{t\in(0,1]}||\boldsymbol{\sigma} - \boldsymbol{\sigma}_h||_{H({\rm div},\Omega)}, 
\quad\text{ and}\quad {\rm err}_{L2} = \max_{t\in(0,1]}||\boldsymbol{\sigma} - \boldsymbol{\sigma}_h||_ {L^2(\Omega)}.
$$

In [ ]:
s_exact = Expression((
    ('sin(pi*x[0]) * sin(pi*x[1]) * sin(2*t)', '0'),
    ('0', '-sin(pi*x[0]) * sin(pi*x[1]) * sin(2*t)')
), degree=4, t=0.0)

mu = Constant(1.0)
f = Expression((
        ('pi*pi*sin(pi*x[0])*sin(pi*x[1])*sin(2*t) + 2*cos(2*t)*sin(pi*x[0])*sin(pi*x[1])/mu', '-pi*pi*cos(pi*x[0])*cos(pi*x[1])*sin(2*t)'),
        ('pi*pi*cos(pi*x[0])*cos(pi*x[1])*sin(2*t)', '-pi*pi*sin(pi*x[0])*sin(pi*x[1])*sin(2*t) - 2*cos(2*t)*sin(pi*x[0])*sin(pi*x[1])/mu')
               ), degree=2, t=0.0, mu=mu)
gD = Expression(('pi*cos(pi*x[0])*sin(pi*x[1])*sin(2*t)', '-pi*sin(pi*x[0])*cos(pi*x[1])*sin(2*t)'), degree=2, t=0)

degree = 1
dt = 0.002
for n in [4, 8, 16]:
    esHd, esL2 = dg_psstokes(n, degree, dt, mu, s_exact, f, gD)
    print('errors: s_Hdiv={:.3e} s_L2={:.3e}'.format(esHd, esL2))

errors: s_Hdiv=4.641e-01 s_L2=4.138e-02
errors: s_Hdiv=2.330e-01 s_L2=8.362e-03
errors: s_Hdiv=1.170e-01 s_L2=2.391e-03


## Weakly-symmetric stress variational formulation (dual form)

We consider the dual formulation of the previous case but as primary variable we consider the physical stress 
$$
\boldsymbol{\sigma}_{\rm phy}(\boldsymbol{u},p) = \mu\boldsymbol{\nabla}_{\rm s}\boldsymbol{u} - p\mathbb{I}_d
= \boldsymbol{\sigma} + {\rm dev} (\boldsymbol{\sigma})^T,
$$
which is a symmetric tensor. The symmetry constraint is enforced weakly in order to ensure the stability of the method without compromising the convergence performance.

In [ ]:
def dg_wsstokes(n, deg, dt, mu, s_exact, f, gD):    
    # 1. generate the mesh and mark the boundaries
    square = Rectangle(Point(0.0, 0.0), Point(1.0, 1.0))
    mesh = generate_mesh(square, n)    
    boundary_markers = MeshFunction('size_t', mesh, mesh.geometric_dimension()-1, 0)

    class top_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 1)

    class right_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 1)

    class other_boundary(SubDomain):
        def inside(self, x, on_boundary):
            return near(x[0], 0) or near(x[1], 0)

    top = top_boundary()
    top.mark(boundary_markers, 1)
    right = right_boundary()
    right.mark(boundary_markers, 2)
    other = other_boundary()
    other.mark(boundary_markers, 3)  
    ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    # 2. finite element space 
    S = TensorElement('DG', mesh.ufl_cell(), deg)
    Q = FiniteElement('DG', mesh.ufl_cell(), deg-1)
    X = FunctionSpace(mesh, MixedElement([S, Q]))

    # 3. problem definition
    s, q = TrialFunctions(X)
    r, p = TestFunctions(X)
    x_old = Function(X)
    s_old, q_old = split(x_old)
    
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    h_avg = (h('+') + h('-'))/2
    alpha = 10.1*deg**2

    a = inner(dev(s), dev(r))/(mu*Constant(dt)) * dx + dot(div(s), div(r)) * dx  \
      - dot(avg(div(s)), jump(r, n)) * dS - dot(jump(s, n), avg(div(r))) * dS \
      - dot(div(s), dot(r, n)) * ds(3) - dot(dot(s, n), div(r)) * ds(3) \
      + alpha/h_avg * dot(jump(s, n), jump(r, n)) * dS \
      + alpha/h*dot(dot(s, n), dot(r, n)) * ds(3) \
      + (p * (s[0,1] - s[1,0]) + q * (r[0,1] - r[1,0])) * dx

    L = (inner(dev(s_old), dev(r))/(mu*Constant(dt)) + inner(f, r)) * dx + dot(gD, dot(r, n)) * (ds(1)+ds(2))
    
    # 4. solution
    A = assemble(a)
    Ainv = LUSolver(A)

    time = 0.0
    max_eHd = 0.0
    max_eL2 = 0.0
    max_eLq = 0.0

    while time < 1.0:
        time += dt
        f.t = time
        gD.t = time
        s_exact.t = time       
        b = assemble(L)
        Ainv.solve(x_old.vector(), b)
        s_old, q_old = x_old.split()
  
        eHd = errornorm(s_exact, s_old, 'Hdiv0')
        eL2 = errornorm(s_exact, s_old, 'L2')
        eLq = norm(q_old, 'L2')
        max_eHd = max(max_eHd, eHd)
        max_eL2 = max(max_eL2, eL2)
        max_eLq = max(max_eLq, eLq)
    
    return max_eHd, max_eL2, max_eLq

The **exact solution** is the same as in the previous convergence test case.

In [ ]:
s_exact = Expression((('sin(pi*x[0]) * sin(pi*x[1]) * sin(2*t)', '0'),
                      ('0', '-sin(pi*x[0]) * sin(pi*x[1]) * sin(2*t)')), degree=5, t=0.0)
mu = Constant(1.0)
f = Expression((
        ('pi*pi*sin(pi*x[0])*sin(pi*x[1])*sin(2*t) + 2*cos(2*t)*sin(pi*x[0])*sin(pi*x[1])/mu', '-pi*pi*cos(pi*x[0])*cos(pi*x[1])*sin(2*t)'),
        ('pi*pi*cos(pi*x[0])*cos(pi*x[1])*sin(2*t)', '-pi*pi*sin(pi*x[0])*sin(pi*x[1])*sin(2*t) - 2*cos(2*t)*sin(pi*x[0])*sin(pi*x[1])/mu')
               ), degree=2, t=0.0, mu=mu)
gD = Expression(('pi*cos(pi*x[0])*sin(pi*x[1])*sin(2*t)', '-pi*sin(pi*x[0])*cos(pi*x[1])*sin(2*t)'), degree=2, t=0)

degree = 1
dt = 0.005
for n in [4, 8, 16]:
    esHd, esL2, eqL2 = dg_wsstokes(n, degree, dt, mu, s_exact, f, gD)
    print('errors: s_Hdiv={:.3e} s_L2={:.3e} q_L2={:.3e}'.format(esHd, esL2, eqL2))

errors: s_Hdiv=4.641e-01 s_L2=4.043e-02 q_L2=1.337e-02
errors: s_Hdiv=2.330e-01 s_L2=9.115e-03 q_L2=4.831e-03
errors: s_Hdiv=1.170e-01 s_L2=4.171e-03 q_L2=3.686e-03
